In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


In [3]:
ticker = "BBAS3.SA"
start_date = "2010-01-01"
end_date = datetime.now().strftime("%Y-%m-%d") #data dinamica

# Baixar os dados históricos
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [4]:
dividends = yf.Ticker(ticker).dividends
data['Dividendos'] = dividends.reindex(data.index).fillna(0)
#Possui os dados diários apenas de: preço de abertura, fechamento, maior  e menor preço do dia, e quantas ações negociadas
data.columns = [col[0] for col in data.columns]
data

,Adj Close,Close,High,Low,Open,Volume,Dividendos
Date,,,,,,,
2010-01-04,5.654894,14.950000,15.000000,14.850000,14.900000,7249400,0.0
2010-01-05,5.598154,14.800000,15.050000,14.595000,15.050000,6129800,0.0
2010-01-06,5.605719,14.820000,15.000000,14.710000,14.745000,5466200,0.0
2010-01-07,5.607610,14.825000,14.855000,14.705000,14.720000,3084400,0.0
2010-01-08,5.639763,14.910000,14.910000,14.735000,14.850000,4077800,0.0
...,...,...,...,...,...,...,...
2024-11-26,25.670000,25.670000,25.780001,25.209999,25.360001,22044000,0.0
2024-11-27,25.219999,25.219999,25.780001,25.110001,25.709999,20884000,0.0
2024-11-28,24.480000,24.480000,25.209999,24.410000,25.190001,26984000,0.0


In [5]:
def calculate_rsi(data, window=14):
    # Calcula a diferença diária nos preços ajustados
    delta = data['Adj Close'].diff()

    # Separa os ganhos e perdas
    gain = delta.clip(lower=0)  # Apenas valores positivos
    loss = -delta.clip(upper=0)  # Apenas valores negativos

    # Cálculo inicial para os primeiros valores (primeiros `window` dias)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Evita divisão por zero (se `avg_loss` for 0, define RSI como 100)
    rs = avg_gain / avg_loss
    rs = rs.replace([float('inf'), float('-inf')], 0)  # Substituir infinitos por 0

    rsi = 100 - (100 / (1 + rs))
    rsi[:window] = None  # Define os primeiros valores como NaN, já que não são confiáveis

    return rsi

def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    ema_short = data['Adj Close'].ewm(span=short_window, adjust=False).mean()
    ema_long = data['Adj Close'].ewm(span=long_window, adjust=False).mean()
    macd = ema_short - ema_long
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

data['RSI'] = calculate_rsi(data)


data['MACD'], data['Signal Line'] = calculate_macd(data)




In [6]:
# Obter o último valor de RSI
ultimo_rsi = data['RSI'].iloc[-1]

# Obter o último valor de MACD
ultimo_macd = data['MACD'].iloc[-1]

# Se quiser obter o último valor da Linha de Sinal (Signal Line)
ultima_signal_line = data['Signal Line'].iloc[-1]


In [7]:
#ordem correta
def calculate_last_rsi(data):
    if isinstance(pd.DataFrame, data):
        ultimo_rsi = data['RSI'].iloc[-1]
        return ultimo_rsi
    else:
        return "Dado inválido"
    
def calculate_last_macd(data):
    if isinstance(pd.DataFrame, data):
        ultimo_macd = data['MACD'].iloc[-1]
        return ultimo_macd
    else:
        return "Dado inválido"

def calculate_last_signal(data):
    if isinstance(pd.DataFrame, data):
        ultima_signal_line = data['Signal Line'].iloc[-1]
        return ultima_signal_line
    else:
        return "Dado inválido"


In [8]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3692 entries, 2010-01-22 to 2024-12-02
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Adj Close    3692 non-null   float64
 1   Close        3692 non-null   float64
 2   High         3692 non-null   float64
 3   Low          3692 non-null   float64
 4   Open         3692 non-null   float64
 5   Volume       3692 non-null   int64  
 6   Dividendos   3692 non-null   float64
 7   RSI          3692 non-null   float64
 8   MACD         3692 non-null   float64
 9   Signal Line  3692 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 317.3 KB


In [9]:
compra = data[data['MACD'] > data['Signal Line']]
compra

,Adj Close,Close,High,Low,Open,Volume,Dividendos,RSI,MACD,Signal Line
Date,,,,,,,,,,
2010-02-02,5.787281,15.300000,15.375000,14.915000,15.085000,10565400,0.0,55.434833,-0.037361,-0.059963
2010-02-03,5.645436,14.925000,15.250000,14.830000,15.110000,5281200,0.0,47.865537,-0.022452,-0.052461
2010-02-04,5.541414,14.650000,14.820000,14.300000,14.800000,5827600,0.0,47.133671,-0.018814,-0.045731
2010-02-05,5.276637,13.950000,14.600000,13.650000,14.430000,12696800,0.0,42.285627,-0.036870,-0.043959
2010-02-09,5.609500,14.830000,15.300000,14.280000,14.300000,10308600,0.0,49.550381,-0.030145,-0.041413
...,...,...,...,...,...,...,...,...,...,...
2024-11-11,25.521816,26.010000,26.100000,25.950001,26.100000,10515100,0.0,38.759710,-0.238687,-0.250825
2024-11-12,25.462942,25.950001,26.090000,25.799999,26.010000,15391500,0.0,35.606007,-0.242267,-0.249114
2024-11-13,25.462942,25.950001,26.070000,25.879999,25.990000,27198100,0.0,34.108409,-0.242312,-0.247753


In [10]:
venda = data[data['MACD'] <= data['Signal Line']]

In [11]:

data_anualbb = [
    {
       'Ano': 2011 ,  
       'P/L': 5.35,
       'PSR': 0.73,
       'P/VP': 1.17,
       'MARGEM LÍQUIDA (%)': 13.61,
       'ROE (%)': 19.91,
       'MARGEM BRUTA (%)': 32.48,
       'MARGEM EBIT (%)': 19.21,
       'P/EBIT': 3.79,
       'ROA (%)': 1.31,
    },
    {
        'Ano': 2012,
        'P/L': 6.52,
        'PSR': 0.78,
        'P/VP': 1.20,
        'MARGEM LÍQUIDA (%)': 11.94,
        'ROE (%)': 16.26,
        'MARGEM BRUTA (%)': 32.05,
        'MARGEM EBIT (%)': 15.94,
        'P/EBIT': 4.89,
        'ROA (%)': 0.99,
    },
    {
        'Ano': 2013,
        'P/L': 6.70,
        'PSR': 0.67,
        'P/VP': 1.00,
        'MARGEM LÍQUIDA (%)': 9.98,
        'ROE (%)': 14.49,
        'MARGEM BRUTA (%)': 28.88,
        'MARGEM EBIT (%)': 12.30,
        'P/EBIT': 5.44,
        'ROA (%)': 0.90,
    },
    {
        'Ano': 2014,
        'P/L': 5.10,
        'PSR': 0.49,
        'P/VP': 0.90,
        'DY (%)': 6.97,
        'MARGEM LÍQUIDA (%)': 8.60,
        'ROE (%)': 14.49,
        'MARGEM BRUTA (%)': 23.13,
        'MARGEM EBIT (%)': 11.33,
        'P/EBIT': 4.36,
        'ROA (%)': 0.93,
    },
    {
        'Ano': 2015,
        'P/L': 2.67,
        'PSR': 0.23,
        'P/VP': 0.61,
        'DY (%)': 11.42,
        'MARGEM LÍQUIDA (%)': 7.71,
        'ROE (%)': 17.04,
        'MARGEM BRUTA (%)': 12.32,
        'MARGEM EBIT (%)': 5.56,
        'P/EBIT': 4.17,
        'ROA (%)': 1.01,
    },
    {
        'Ano': 2016,
        'P/L': 11.45,
        'PSR': 0.46,
        'P/VP': 1.06,
        'DY (%)': 2.65,
        'MARGEM LÍQUIDA (%)': 3.99,
        'MARGEM EBIT (%)': 6.18,
        'ROE (%)': 8.14,
        'MARGEM BRUTA (%)': 17.58,
        'P/EBIT': 7.39,
        'ROA (%)': 0.51,
    },
    {
        'Ano': 2017,
        'P/L': 8.58,
        'PSR': 0.62,
        'P/VP': 1.04,
        'DY (%)': 2.96,
        'MARGEM LÍQUIDA (%)': 7.23,
        'MARGEM EBIT (%)': 10.83,
        'ROE (%)': 10.92,
        'MARGEM BRUTA (%)': 21.31,
        'P/EBIT': 5.72,
        'ROA (%)': 0.79,
    },
    {
        'Ano': 2018,
        'P/L': 9.63,
        'PSR': 1.11,
        'P/VP': 1.45,
        'DY (%)': 3.26,
        'MARGEM LÍQUIDA (%)': 11.50,
        'MARGEM EBIT (%)': 16.98,
        'ROE (%)': 13.57,
        'MARGEM BRUTA (%)': 31.68,
        'P/EBIT': 6.53,
        'ROA (%)': 0.99,
    },
    {
        'Ano': 2019,
        'P/L': 9.23,
        'PSR': 1.23,
        'P/VP': 1.53,
        'DY (%)': 4.84,
        'MARGEM LÍQUIDA (%)': 13.32,
        'MARGEM EBIT (%)': 9.05,
        'ROE (%)': 15.22,
        'MARGEM BRUTA (%)': 27.50,
        'P/EBIT': 13.59,
        'ROA (%)': 1.07,
    },
    {
        'Ano': 2020,
        'P/L': 8.36,
        'PSR': 1.13,
        'P/VP': 0.95,
        'DY (%)': 3.81,
        'MARGEM LÍQUIDA (%)': 13.47,
        'MARGEM EBIT (%)': 11.73,
        'ROE (%)': 10.63,
        'MARGEM BRUTA (%)': 56.18,
        'P/EBIT': 9.60,
        'ROA (%)': 0.75,
    },
    {
        'Ano': 2021,
        'P/L': 4.19,
        'PSR': 0.66,
        'P/VP': 0.58,
        'DY (%)': 7.86,
        'MARGEM LÍQUIDA (%)': 15.66,
        'MARGEM EBIT (%)': 19.05,
        'ROE (%)': 13.82,
        'MARGEM BRUTA (%)': 47.38,
        'P/EBIT': 3.45,
        'ROA (%)': 0.99,
    },
    {
        'Ano': 2022,
        'P/L': 3.32,
        'PSR': 0.42,
        'P/VP': 0.62,
        'DY (%)': 12.01,
        'MARGEM LÍQUIDA (%)': 12.68,
        'MARGEM EBIT (%)': 16.26,
        'ROE (%)': 18.79,
        'MARGEM BRUTA (%)': 31.43,
        'P/EBIT': 2.59,
        'ROA (%)': 1.49,
    },
    {
        'Ano': 2023,
        'P/L': 4.79,
        'PSR': 0.60,
        'P/VP': 0.94,
        'DY (%)': 8.26,
        'MARGEM LÍQUIDA (%)': 12.49,
        'MARGEM EBIT (%)': 15.50,
        'ROE (%)': 19.60,
        'MARGEM BRUTA (%)': 33.43,
        'P/EBIT': 3.86,
        'ROA (%)': 1.54,
    },
    {
        'Ano': 2024,
        'P/L': 4.56,
        'PSR': 0.55,
        'P/VP': 0.82,
        'DY (%)': 8.34,
        'MARGEM LÍQUIDA (%)': 12.13,
        'MARGEM EBIT (%)': 12.73,
        'ROE (%)': 17.87,
        'MARGEM BRUTA (%)': 37.76,
        'P/EBIT': 4.35,
        'ROA (%)': 1.37,
    }
]

# Criar o DataFrame
df = pd.DataFrame(data_anualbb)

# Definir a ordem das colunas (opcional)
colunas = ['Ano', 'P/L', 'PSR', 'P/VP', 'DY (%)', 'MARGEM BRUTA (%)', 'MARGEM EBIT (%)', 'MARGEM LÍQUIDA (%)', 'P/EBIT', 'ROE (%)', 'ROA (%)']
df = df[colunas]



# Exibir o DataFrame
print(df)


     Ano    P/L   PSR  P/VP  DY (%)  MARGEM BRUTA (%)  MARGEM EBIT (%)  \
0   2011   5.35  0.73  1.17     NaN             32.48            19.21   
1   2012   6.52  0.78  1.20     NaN             32.05            15.94   
2   2013   6.70  0.67  1.00     NaN             28.88            12.30   
3   2014   5.10  0.49  0.90    6.97             23.13            11.33   
4   2015   2.67  0.23  0.61   11.42             12.32             5.56   
5   2016  11.45  0.46  1.06    2.65             17.58             6.18   
6   2017   8.58  0.62  1.04    2.96             21.31            10.83   
7   2018   9.63  1.11  1.45    3.26             31.68            16.98   
8   2019   9.23  1.23  1.53    4.84             27.50             9.05   
9   2020   8.36  1.13  0.95    3.81             56.18            11.73   
10  2021   4.19  0.66  0.58    7.86             47.38            19.05   
11  2022   3.32  0.42  0.62   12.01             31.43            16.26   
12  2023   4.79  0.60  0.94    8.26   

In [12]:


data = [
{
   'Ano': 2011 ,  
   'P/L': 5.35,
   'PSR': 0.73,
   'P/VP': 1.17,
   'MARGEM LÍQUIDA': 13.61,
   'ROE': 19.91,
   'MARGEM BRUTA': 32.48,
   'MARGEM EBIT': 19.21,
   'P/EBIT': 3.79 ,
   'ROA': 1.31,
},
{
    'Ano': 2012,
    'P/L': 6.52,
    'PSR': 0.78,
    'P/VP': 1.2,
    'MARGEM LÍQUIDA': 11.94,
    'ROE': 16.26,
    'MARGEM BRUTA': 32.05,
    'MARGEM EBIT': 15.94,
    'P/EBIT': 4.89,
    'ROA': 0.99,
},
{
    'Ano': 2013,
    'P/L': 6.7,
    'PSR': 0.67,
    'P/VP': 1,
    'MARGEM LÍQUIDA': 9.98,
    'ROE': 14.49,
    'MARGEM BRUTA': 28.88,
    'MARGEM EBIT': 12.3,
    'P/EBIT': 5.44,
    'ROA': 0.9,
},
{
    'Ano': 2014,
    'P/L': 5.1,
    'PSR': 0.49,
    'P/VP': 0.9,
    'DY': 6.97,
    'MARGEM LÍQUIDA': 8.6,
    'ROE': 14.49,
    'MARGEM BRUTA': 23.13,
    'MARGEM EBIT': 11.33,
    'P/EBIT': 4.36,
    'ROA': 0.93,

},
{
    'Ano': 2015,
    'P/L': 2.67,
    'PSR': 0.23,
    'P/VP': 0.61,
    'DY': 11.42,
    'PAYOUT': 30.54,
    'MARGEM LÍQUIDA': 7.71,
    'ROE': 17.04,
    'MARGEM BRUTA': 12.32,
    'MARGEM EBIT': 5.56,
    'MARGEM EBITDA': 5.56,
    'P/EBIT': 4.17,
    'ROA': 1.01,
},
{
    'Ano': 2016,
    'P/L': 11.45,
    'PSR': 0.46,
    'P/VP': 1.06,
    'DY': 2.65,
    'MARGEM LÍQUIDA': 3.99,
    'MARGEM EBIT': 6.18,
    'ROE': 8.14,
    'MARGEM BRUTA': 17.58,
    'P/EBIT': 7.39,
    'ROA': 0.51,
},
{
    'Ano': 2017,
    'P/L': 8.58,
    'PSR': 0.62,
    'P/VP': 1.04,
    'DY': 2.96,
    'MARGEM LÍQUIDA': 7.23,
    'MARGEM EBIT': 10.83,
    'ROE': 10.92,
    'MARGEM BRUTA': 21.31,
    'P/EBIT': 5.72,
    'ROA': 0.79,
},
{
    'Ano': 2018,
    'P/L': 9.63,
    'PSR': 1.11,
    'P/VP': 1.45,
    'DY': 3.26,
    'MARGEM LÍQUIDA': 11.5,
    'MARGEM EBIT': 16.98,
    'ROE': 13.57,
    'MARGEM BRUTA': 31.68,
    'P/EBIT': 6.53,
    'ROA': 0.99,
},
{
    'Ano': 2019,
    'P/L': 9.23,
    'PSR': 1.23,
    'P/VP': 1.53,
    'DY': 4.84,
    'MARGEM LÍQUIDA': 13.32,
    'MARGEM EBIT': 9.05,
    'ROE': 15.22,
    'MARGEM BRUTA': 27.5,
    'P/EBIT': 13.59,
    'ROA': 1.07,
},
{
    'Ano': 2020,
    'P/L': 8.36,
    'PSR': 1.13,
    'P/VP': 0.95,
    'DY': 3.81,
    'MARGEM LÍQUIDA': 13.47,
    'MARGEM EBIT': 11.73,
    'ROE': 10.63,
    'MARGEM BRUTA': 56.18,
    'P/EBIT': 9.6,
    'ROA': 0.75,
},
{
    'Ano': 2021,
    'P/L': 4.19,
    'PSR': 0.66,
    'P/VP': 0.58,
    'DY': 7.86,
    'MARGEM LÍQUIDA': 15.66,
    'MARGEM EBIT': 19.05,
    'ROE': 13.82,
    'MARGEM BRUTA': 47.38,
    'P/EBIT': 3.45,
    'ROA': 0.99,
},
{
    'Ano': 2022,
    'P/L': 3.32,
    'PSR': 0.42,
    'P/VP': 0.62,
    'DY': 12.01,
    'MARGEM LÍQUIDA': 12.68,
    'MARGEM EBIT': 16.26,
    'ROE': 18.79,
    'MARGEM BRUTA': 31.43,
    'P/EBIT': 2.59,
    'ROA': 1.49,
},
{
    'Ano': 2023,
    'P/L': 4.79,
    'PSR': 0.6,
    'P/VP': 0.94,
    'DY': 8.26,
    'MARGEM LÍQUIDA': 12.49,
    'MARGEM EBIT': 15.5,
    'ROE': 19.6,
    'MARGEM BRUTA': 33.43,
    'P/EBIT': 3.86,
    'ROA': 1.54,
},
{
    'Ano': 2024,
    'P/L': 4.56,
    'PSR': 0.55,
    'P/VP': 0.82,
    'DY': 8.34,
    'MARGEM LÍQUIDA': 12.13,
    'MARGEM EBIT': 12.73,
    'ROE': 17.87,
    'MARGEM BRUTA': 37.76,
    'P/EBIT': 4.35,
    'ROA': 1.37,
}


SyntaxError: incomplete input (1188814537.py, line 183)

In [ ]:
url = "https://investidor10.com.br/acoes/bbas3/"
response = requests.get(url)
response

<Response [403]>